In [50]:
# Import requests library
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [51]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [52]:
import getpass
id_secret=getpass.getpass()

········


In [53]:
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': '63a66212489e42c085b1e312ba6098ac',
    'client_secret': id_secret,
})

In [54]:
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

In [55]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [56]:
BASE_URL = 'https://api.spotify.com/v1/'
playlist = requests.get(BASE_URL + 'playlists/5XKt7CsZlmWcDRsxcsUh3m/tracks', headers=headers)
#playlist.json()

In [57]:
playlist_df = pd.DataFrame(playlist.json())

In [58]:
playlist_df.head()

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/5XKt7CsZl...,"{'added_at': '2014-10-03T01:52:20Z', 'added_by...",100,https://api.spotify.com/v1/playlists/5XKt7CsZl...,0,None,130
1,https://api.spotify.com/v1/playlists/5XKt7CsZl...,"{'added_at': '2014-10-03T01:52:47Z', 'added_by...",100,https://api.spotify.com/v1/playlists/5XKt7CsZl...,0,None,130
2,https://api.spotify.com/v1/playlists/5XKt7CsZl...,"{'added_at': '2014-10-03T01:52:59Z', 'added_by...",100,https://api.spotify.com/v1/playlists/5XKt7CsZl...,0,None,130
3,https://api.spotify.com/v1/playlists/5XKt7CsZl...,"{'added_at': '2014-10-03T01:53:15Z', 'added_by...",100,https://api.spotify.com/v1/playlists/5XKt7CsZl...,0,None,130
4,https://api.spotify.com/v1/playlists/5XKt7CsZl...,"{'added_at': '2014-10-03T01:53:19Z', 'added_by...",100,https://api.spotify.com/v1/playlists/5XKt7CsZl...,0,None,130


In [59]:
total_songs=playlist_df.loc[0,:][6]

In [60]:
total_songs

130

In [61]:
number_songs=0

In [62]:
artists=[]
songs=[]
track_ids=[]

In [63]:
# read actual request (100 songs) and additional results (if exists)
while number_songs < total_songs:
    playlist_norm = json_normalize(playlist_df['items'])
 #   playlist_norm.columns
    for item in range(len(playlist_norm['track.name'])):
        artists.append(playlist_norm['track.artists'][item][0]['name'])
        songs.append(playlist_norm['track.name'][item])
        track_ids.append(playlist_norm['track.id'][item])
    if (total_songs-number_songs)<100:
        break
    playlist = requests.get(playlist_df.loc[0,:][3], headers=headers)
    playlist_df = pd.DataFrame(playlist.json())
    number_songs+=100

C:\Users\Usuario\AppData\Local\Temp/ipykernel_1924/2684746459.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  playlist_norm = json_normalize(playlist_df['items'])


In [66]:
len(songs)

130

In [67]:
repository = pd.DataFrame({"artist":artists,
                       "song":songs,
                           "track_id": track_ids
                          })

In [68]:
repository.head()

,artist,song,track_id
0,Prince,When Doves Cry,6ZB15yAQctgJLoM3zz2DWA
1,Irene Cara,"Fame - From ""Fame"" Original Motion Picture Sou...",45StDg3fxr2BEc1AyyvtTJ
2,Bananarama,Venus,6KqWBiEZxRmPPGDc0PkRoV
3,a-ha,Take On Me,1nbMkRhfldXcmCyB83x8lK
4,Madonna,Material Girl,7F2b4lHBr8V7JzNbN1ipCe


In [69]:
len(repository)

130

In [70]:
repository.to_csv('spotify_playlist.csv')